In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import warnings



# Change the html tags according your convinience 

In [17]:
# Tata cars df


# function to extract car name
def get_title(soup):
    try:
        title = soup.find('h1', attrs={'class': 'thcHeading'}).string.strip()
    except AttributeError:
        title = ""
    return title

# function to extract ex-showroom price
def get_price(soup):
    try:
        price = soup.find('div', class_='price').text.split('*')[0].strip()
    except AttributeError:
        price = ""
    return price

# function to extract product rating
def get_rating(soup):
    try:
        rating = soup.find('span', attrs={'class': 'ratingStarNew'}).string.strip()
    except AttributeError:
        rating = ""
    return rating

# function to extract number of user reviews
def get_review_count(soup):
    try:
        review_count = soup.find('span', attrs={'class': 'reviews'}).string.strip()
    except AttributeError:
        review_count = ""               
    return review_count





def get_onroadprice(link, HEADERS):
    """
    Fetches the on-road price for the given car link.
    """
    try:
        # Determine the correct URL structure
        if link in ('https://www.cardekho.com/carmodels/Tata/Tata_Tiago'):
            URL = 'https://www.cardekho.com/tata-tiago/car-price-in-new-delhi.htm'
        elif link in ('https://www.cardekho.com/carmodels/Tata/Tata_Safari'):
            URL = 'https://www.cardekho.com/tata-safari/car-price-in-new-delhi.htm'
        else:
            URL = link + '/price-in-new-delhi'            

        # Fetch the webpage content
        webpage = requests.get(URL, headers=HEADERS)
        webpage.raise_for_status()
        soup = BeautifulSoup(webpage.content, 'html.parser')

        # Extract the on-road price
        price_element = soup.find('td', attrs={'class': 'gsc_col-xs-4 gsc_col-md-4'})    # class="gsc_col-xs-4 gsc_col-md-4"
        if price_element:
            return price_element.text.strip()
        else:
            return "On-road price not found."

    except Exception as e:
        return f"Error occurred: {e}"

def starting_emi(link, HEADERS):

        try:
            if link in ('https://www.cardekho.com/carmodels/Tata/Tata_Tiago'):
                URL = 'https://www.cardekho.com/tata-tiago/car-price-in-new-delhi.htm'
            elif link in ('https://www.cardekho.com/carmodels/Tata/Tata_Safari'):
                URL = 'https://www.cardekho.com/tata-safari/car-price-in-new-delhi.htm'
            else:
                URL = link + '/price-in-new-delhi'
    
            # Fetch the webpage content
            webpage = requests.get(URL, headers=HEADERS)
            webpage.raise_for_status()
            soup = BeautifulSoup(webpage.content, 'html.parser')
    
            # Extract the starting EMI
            emi_element = soup.find('span', attrs={'class': 'bold'})
            if emi_element:
                return emi_element.text.strip()
            else:
                return "Starting EMI not found."

        except Exception as e:
            return f"Error occurred: {e}"


In [20]:


if __name__ == '__main__':
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en:q=0.5'
    }
    URL = "https://www.cardekho.com/cars/Tata"

    # HTTP request
    webpage = requests.get(URL, headers=HEADERS)
    if webpage.status_code == 200:
        print("Request successful!")
    else:
        print(f"Request failed with status code: {webpage.status_code}")
        exit()

    soup = BeautifulSoup(webpage.content, "html.parser")
    links = soup.find_all("div", attrs={'class': 'gsc_col-sm-12 gsc_col-xs-12 gsc_col-md-8 listView holder posS'})
    links_list = [link.find("a", href=True).get('href') for link in links if link.find("a", href=True)]

    print(f"Found {len(links_list)} links.")

    d = {"title": [], "ex-showroom price": [],"on road price":[],"starting EMI":[], "rating": [], "reviews": []}

    for i, link in enumerate(links_list):
        print(f"Processing link {i+1}/{len(links_list)}: {link}")
        try:
            new_webpage = requests.get(link, headers=HEADERS)
            if new_webpage.status_code != 200:
                print(f"Failed to fetch {link}: Status {new_webpage.status_code}")
                continue
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")
            d['title'].append(get_title(new_soup))
            d['ex-showroom price'].append(get_price(new_soup))
            d['on road price'].append(get_onroadprice(link,HEADERS))
            d['starting EMI'].append(starting_emi(link,HEADERS))
            d['rating'].append(get_rating(new_soup))
            d['reviews'].append(get_review_count(new_soup))
            
            
        except Exception as e:
            print(f"Error processing link {link}: {e}")

    
    for key, value in d.items():
        print(f"Key: {key}, Length: {len(value)}")
    
    max_length = max(len(v) for v in d.values())  # Find the maximum length

    # Ensure all lists are of the same length
    for key in d:
        current_length = len(d[key])
        if current_length < max_length:
            d[key].extend([None] * (max_length - current_length))  # Pad with None
        elif current_length > max_length:
            d[key] = d[key][:max_length]  # Trim extra elements

    Tata_df = pd.DataFrame.from_dict(d)
    print(Tata_df.head())
    Tata_df['title'] = Tata_df['title'].replace('', np.nan)
    Tata_df = Tata_df.dropna(subset=['title'])

    Tata_df.to_csv("cars_df.csv", encoding='utf-8', index=False)
    print("File saved as CSV successfully!")
    
    print("File saved successfully!")

# Save to Excel
   



Request successful!
Found 16 links.
Processing link 1/16: https://www.cardekho.com/tata/punch
Processing link 2/16: https://www.cardekho.com/tata/nexon
Processing link 3/16: https://www.cardekho.com/tata/curvv
Processing link 4/16: https://www.cardekho.com/tata/harrier
Processing link 5/16: https://www.cardekho.com/carmodels/Tata/Tata_Tiago
Processing link 6/16: https://www.cardekho.com/carmodels/Tata/Tata_Safari
Processing link 7/16: https://www.cardekho.com/tata/altroz
Processing link 8/16: https://www.cardekho.com/tata/curvv-ev
Processing link 9/16: https://www.cardekho.com/tata/nexon-ev
Processing link 10/16: https://www.cardekho.com/tata/tigor
Processing link 11/16: https://www.cardekho.com/tata/punch-ev
Processing link 12/16: https://www.cardekho.com/tata/tiago-ev
Processing link 13/16: https://www.cardekho.com/tata/altroz-racer
Processing link 14/16: https://www.cardekho.com/tata/yodha-pickup
Processing link 15/16: https://www.cardekho.com/tata/tigor-ev
Processing link 16/16: ht

In [25]:
# maruti suzuki df




# function to extract car name
def get_title(soup):
    try:
        title = soup.find('h1', attrs = {'class':'thcHeading'}).string.strip()
    except AttributeError:
        title = ""
    return title

# function to extract ex-showroom price
def get_price(soup):
    try:
        price = soup.find('div', class_='price').text.split('*')[0].strip()
    except AttributeError:
        price = ""
    return price

# function to extract product rating
def get_rating(soup):
    try:
        rating = soup.find('span',attrs = {'class':'ratingStarNew'}).string.strip()
    except AttributeError:
        rating = ""
    return rating

# function to extract number of user reviews
def get_review_count(soup):
    try:
        review_count = soup.find('span',attrs = {'class':'reviews'}).string.strip()
    except AttributeError:
        review_count = ""               
    return review_count


def get_onroadprice(link, HEADERS):
    try:
        
        if link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ertiga':
            URL = 'https://www.cardekho.com/maruti-ertiga/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Grand_Vitara':
            URL = 'https://www.cardekho.com/maruti-grand-vitara/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno':
            URL = 'https://www.cardekho.com/maruti-baleno/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio':
            URL = 'https://www.cardekho.com/maruti-celerio/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco':
            URL = 'https://www.cardekho.com/maruti-eeco/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis':
            URL = 'https://www.cardekho.com/maruti-ignis/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz':
            URL = 'https://www.cardekho.com/maruti-ciaz/car-price-in-new-delhi.htm'
        else:
            URL = link + '/price-in-new-delhi'            

        # Fetch the webpage content
        webpage = requests.get(URL, headers=HEADERS)
        webpage.raise_for_status()
        soup = BeautifulSoup(webpage.content, 'html.parser')

        # Extract the on-road price
        price_element = soup.find('td', attrs={'class': 'gsc_col-xs-4 gsc_col-md-4'})    # class="gsc_col-xs-4 gsc_col-md-4"
        if price_element:
            return price_element.text.strip()
        else:
            return "On-road price not found."

    except Exception as e:
        return f"Error occurred: {e}"

def starting_emi(link, HEADERS):

        try:
            if link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ertiga':
                URL = 'https://www.cardekho.com/maruti-ertiga/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Grand_Vitara':
                URL = 'https://www.cardekho.com/maruti-grand-vitara/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno':
                URL = 'https://www.cardekho.com/maruti-baleno/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio':
                URL = 'https://www.cardekho.com/maruti-celerio/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco':
                URL = 'https://www.cardekho.com/maruti-eeco/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis':
                URL = 'https://www.cardekho.com/maruti-ignis/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz':
                URL = 'https://www.cardekho.com/maruti-ciaz/car-price-in-new-delhi.htm'
            else:
                URL = link + '/price-in-new-delhi'       
    
            # Fetch the webpage content
            webpage = requests.get(URL, headers=HEADERS)
            webpage.raise_for_status()
            soup = BeautifulSoup(webpage.content, 'html.parser')
    
            # Extract the starting EMI
            emi_element = soup.find('span', attrs={'class': 'bold'})
            if emi_element:
                return emi_element.text.strip()
            else:
                return "Starting EMI not found."

        except Exception as e:
            return f"Error occurred: {e}"




In [26]:

if __name__ == '__main__':
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en:q=0.5'
    }
    URL = 'https://www.cardekho.com/maruti-suzuki-cars'

    # HTTP request
    webpage = requests.get(URL, headers=HEADERS)
    if webpage.status_code == 200:
        print("Request successful!")
    else:
        print(f"Request failed with status code: {webpage.status_code}")
        exit()

    soup = BeautifulSoup(webpage.content, "html.parser")
    links = soup.find_all('div',attrs={'class':'gsc_col-sm-12 gsc_col-xs-12 gsc_col-md-8 listView holder posS'})
    links_list = [link.find("a", href=True).get('href') for link in links if link.find("a", href=True)]

    print(f"Found {len(links_list)} links.")

    d = {"title": [], "ex-showroom price": [],"on road price":[],"starting EMI":[], "rating": [], "reviews": []}  #"on road price":[],"starting EMI":[],

    for i, link in enumerate(links_list):
        print(f"Processing link {i+1}/{len(links_list)}: {link}")
        try:
            new_webpage = requests.get(link, headers=HEADERS)
            if new_webpage.status_code != 200:
                print(f"Failed to fetch {link}: Status {new_webpage.status_code}")
                continue
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")
            d['title'].append(get_title(new_soup))
            d['ex-showroom price'].append(get_price(new_soup))
            d['on road price'].append(get_onroadprice(link,HEADERS))
            d['starting EMI'].append(starting_emi(link,HEADERS))
            d['rating'].append(get_rating(new_soup))
            d['reviews'].append(get_review_count(new_soup))
            
            
        except Exception as e:
            print(f"Error processing link {link}: {e}")

    
    for key, value in d.items():
        print(f"Key: {key}, Length: {len(value)}")
    
    max_length = max(len(v) for v in d.values())  # Find the maximum length

    # Ensure all lists are of the same length
    for key in d:
        current_length = len(d[key])
        if current_length < max_length:
            d[key].extend([None] * (max_length - current_length))  # Pad with None
        elif current_length > max_length:
            d[key] = d[key][:max_length]  # Trim extra elements

    Tata_df = pd.DataFrame.from_dict(d)
    print(Tata_df.head())
    Tata_df['title'] = Tata_df['title'].replace('', np.nan)
    Tata_df = Tata_df.dropna(subset=['title'])

    Tata_df.to_csv("maruti_df.csv", encoding='utf-8', index=False)
    print("File saved as CSV successfully!")
    
    print("File saved successfully!")


   



Request successful!
Found 23 links.
Processing link 1/23: https://www.cardekho.com/maruti/dzire
Processing link 2/23: https://www.cardekho.com/maruti/swift
Processing link 3/23: https://www.cardekho.com/carmodels/Maruti/Maruti_Ertiga
Processing link 4/23: https://www.cardekho.com/maruti/fronx
Processing link 5/23: https://www.cardekho.com/maruti/brezza
Processing link 6/23: https://www.cardekho.com/carmodels/Maruti/Maruti_Grand_Vitara
Processing link 7/23: https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno
Processing link 8/23: https://www.cardekho.com/maruti/wagon-r
Processing link 9/23: https://www.cardekho.com/maruti/alto-k10
Processing link 10/23: https://www.cardekho.com/maruti/jimny
Processing link 11/23: https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio
Processing link 12/23: https://www.cardekho.com/maruti/xl6
Processing link 13/23: https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco
Processing link 14/23: https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis
Pro

In [30]:



# mahindra df





# function to extract car name
def get_title(soup):
    try:
        title = soup.find('h1', attrs = {'class':'thcHeading'}).string.strip()
    except AttributeError:
        title = ""
    return title

# function to extract ex-showroom price
def get_price(soup):
    try:
        price = soup.find('div', class_='price').text.split('*')[0].strip()
    except AttributeError:
        price = ""
    return price

# function to extract product rating
def get_rating(soup):
    try:
        rating = soup.find('span',attrs = {'class':'ratingStarNew'}).string
    except AttributeError:
        rating = ""
    return rating

# function to extract number of user reviews
def get_review_count(soup):
    try:
        review_count = soup.find('span',attrs = {'class':'reviews'}).string.strip()
    except AttributeError:
        review_count = ""               
    return review_count


def get_onroadprice(link, HEADERS):
    try:
        
        if link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ertiga':
            URL = 'https://www.cardekho.com/maruti-ertiga/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Grand_Vitara':
            URL = 'https://www.cardekho.com/maruti-grand-vitara/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno':
            URL = 'https://www.cardekho.com/maruti-baleno/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio':
            URL = 'https://www.cardekho.com/maruti-celerio/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco':
            URL = 'https://www.cardekho.com/maruti-eeco/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis':
            URL = 'https://www.cardekho.com/maruti-ignis/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz':
            URL = 'https://www.cardekho.com/maruti-ciaz/car-price-in-new-delhi.htm'
        else:
            URL = link + '/price-in-new-delhi'            

        # Fetch the webpage content
        webpage = requests.get(URL, headers=HEADERS)
        webpage.raise_for_status()
        soup = BeautifulSoup(webpage.content, 'html.parser')

        # Extract the on-road price
        price_element = soup.find('td', attrs={'class': 'gsc_col-xs-4 gsc_col-md-4'})    # class="gsc_col-xs-4 gsc_col-md-4"
        if price_element:
            return price_element.text.strip()
        else:
            return "On-road price not found."

    except Exception as e:
        return f"Error occurred: {e}"

def starting_emi(link, HEADERS):

        try:
            if link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ertiga':
                URL = 'https://www.cardekho.com/maruti-ertiga/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Grand_Vitara':
                URL = 'https://www.cardekho.com/maruti-grand-vitara/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno':
                URL = 'https://www.cardekho.com/maruti-baleno/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio':
                URL = 'https://www.cardekho.com/maruti-celerio/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco':
                URL = 'https://www.cardekho.com/maruti-eeco/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis':
                URL = 'https://www.cardekho.com/maruti-ignis/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz':
                URL = 'https://www.cardekho.com/maruti-ciaz/car-price-in-new-delhi.htm'
            else:
                URL = link + '/price-in-new-delhi'       
    
            # Fetch the webpage content
            webpage = requests.get(URL, headers=HEADERS)
            webpage.raise_for_status()
            soup = BeautifulSoup(webpage.content, 'html.parser')
    
            # Extract the starting EMI
            emi_element = soup.find('span', attrs={'class': 'bold'})
            if emi_element:
                return emi_element.text.strip()
            else:
                return "Starting EMI not found."

        except Exception as e:
            return f"Error occurred: {e}"




In [33]:
if __name__ == '__main__':
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en:q=0.5'
    }
    URL = 'https://www.cardekho.com/cars/Mahindra'

    # HTTP request
    webpage = requests.get(URL, headers=HEADERS)
    if webpage.status_code == 200:
        print("Request successful!")
    else:
        print(f"Request failed with status code: {webpage.status_code}")
        exit()

    soup = BeautifulSoup(webpage.content, "html.parser")
    links = soup.find_all('div',attrs={'class':'gsc_col-sm-12 gsc_col-xs-12 gsc_col-md-8 listView holder posS'})
    links_list = [link.find("a", href=True).get('href') for link in links if link.find("a", href=True)]

    print(f"Found {len(links_list)} links.")

    d = {"title": [], "ex-showroom price": [],"on road price":[],"starting EMI":[], "rating": [], "reviews": []}  #"on road price":[],"starting EMI":[],

    for i, link in enumerate(links_list):
        print(f"Processing link {i+1}/{len(links_list)}: {link}")
        try:
            new_webpage = requests.get(link, headers=HEADERS)
            if new_webpage.status_code != 200:
                print(f"Failed to fetch {link}: Status {new_webpage.status_code}")
                continue
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")
            d['title'].append(get_title(new_soup))
            d['ex-showroom price'].append(get_price(new_soup))
            d['on road price'].append(get_onroadprice(link,HEADERS))
            d['starting EMI'].append(starting_emi(link,HEADERS))
            d['rating'].append(get_rating(new_soup))
            d['reviews'].append(get_review_count(new_soup))
            
            
        except Exception as e:
            print(f"Error processing link {link}: {e}")

    
    for key, value in d.items():
        print(f"Key: {key}, Length: {len(value)}")
    
    max_length = max(len(v) for v in d.values())  # Find the maximum length

    # Ensure all lists are of the same length
    for key in d:
        current_length = len(d[key])
        if current_length < max_length:
            d[key].extend([None] * (max_length - current_length))  # Pad with None
        elif current_length > max_length:
            d[key] = d[key][:max_length]  # Trim extra elements

    Tata_df = pd.DataFrame.from_dict(d)
    print(Tata_df.head())
    Tata_df['title'] = Tata_df['title'].replace('', np.nan)
    Tata_df = Tata_df.dropna(subset=['title'])

    Tata_df.to_csv("mahindra_df.csv", encoding='utf-8', index=False)
    print("File saved as CSV successfully!")
    
    print("File saved successfully!")


   



Request successful!
Found 16 links.
Processing link 1/16: https://www.cardekho.com/mahindra/scorpio-n
Processing link 2/16: https://www.cardekho.com/mahindra/thar-roxx
Processing link 3/16: https://www.cardekho.com/mahindra/bolero
Processing link 4/16: https://www.cardekho.com/mahindra/xuv700
Processing link 5/16: https://www.cardekho.com/mahindra/scorpio
Processing link 6/16: https://www.cardekho.com/mahindra/thar
Processing link 7/16: https://www.cardekho.com/mahindra/xuv-3xo
Processing link 8/16: https://www.cardekho.com/mahindra/be-6
Processing link 9/16: https://www.cardekho.com/mahindra/xev-9e
Processing link 10/16: https://www.cardekho.com/mahindra/bolero-neo
Processing link 11/16: https://www.cardekho.com/mahindra/xuv400-ev
Processing link 12/16: https://www.cardekho.com/mahindra/bolero-camper
Processing link 13/16: https://www.cardekho.com/mahindra/bolero-pik-up-extra-long
Processing link 14/16: https://www.cardekho.com/mahindra/bolero-maxi-truck-plus
Processing link 15/16: ht

In [34]:
# honda car df
# maruti suzuki df




# function to extract car name
def get_title(soup):
    try:
        title = soup.find('h1', attrs = {'class':'thcHeading'}).string.strip()
    except AttributeError:
        title = ""
    return title

# function to extract ex-showroom price
def get_price(soup):
    try:
        price = soup.find('div', class_='price').text.split('*')[0].strip()
    except AttributeError:
        price = ""
    return price

# function to extract product rating
def get_rating(soup):
    try:
        rating = soup.find('span',attrs = {'class':'ratingStarNew'}).string.strip()
    except AttributeError:
        rating = ""
    return rating

# function to extract number of user reviews
def get_review_count(soup):
    try:
        review_count = soup.find('span',attrs = {'class':'reviews'}).string.strip()
    except AttributeError:
        review_count = ""               
    return review_count


def get_onroadprice(link, HEADERS):
    try:
        
        if link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ertiga':
            URL = 'https://www.cardekho.com/maruti-ertiga/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Grand_Vitara':
            URL = 'https://www.cardekho.com/maruti-grand-vitara/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno':
            URL = 'https://www.cardekho.com/maruti-baleno/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio':
            URL = 'https://www.cardekho.com/maruti-celerio/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco':
            URL = 'https://www.cardekho.com/maruti-eeco/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis':
            URL = 'https://www.cardekho.com/maruti-ignis/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz':
            URL = 'https://www.cardekho.com/maruti-ciaz/car-price-in-new-delhi.htm'
        else:
            URL = link + '/price-in-new-delhi'            

        # Fetch the webpage content
        webpage = requests.get(URL, headers=HEADERS)
        webpage.raise_for_status()
        soup = BeautifulSoup(webpage.content, 'html.parser')

        # Extract the on-road price
        price_element = soup.find('td', attrs={'class': 'gsc_col-xs-4 gsc_col-md-4'})    # class="gsc_col-xs-4 gsc_col-md-4"
        if price_element:
            return price_element.text.strip()
        else:
            return "On-road price not found."

    except Exception as e:
        return f"Error occurred: {e}"

def starting_emi(link, HEADERS):

        try:
            if link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ertiga':
                URL = 'https://www.cardekho.com/maruti-ertiga/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Grand_Vitara':
                URL = 'https://www.cardekho.com/maruti-grand-vitara/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno':
                URL = 'https://www.cardekho.com/maruti-baleno/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio':
                URL = 'https://www.cardekho.com/maruti-celerio/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco':
                URL = 'https://www.cardekho.com/maruti-eeco/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis':
                URL = 'https://www.cardekho.com/maruti-ignis/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz':
                URL = 'https://www.cardekho.com/maruti-ciaz/car-price-in-new-delhi.htm'
            else:
                URL = link + '/price-in-new-delhi'       
    
            # Fetch the webpage content
            webpage = requests.get(URL, headers=HEADERS)
            webpage.raise_for_status()
            soup = BeautifulSoup(webpage.content, 'html.parser')
    
            # Extract the starting EMI
            emi_element = soup.find('span', attrs={'class': 'bold'})
            if emi_element:
                return emi_element.text.strip()
            else:
                return "Starting EMI not found."

        except Exception as e:
            return f"Error occurred: {e}"





In [6]:
if __name__ == '__main__':
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en:q=0.5'
    }
    URL = 'https://www.cardekho.com/cars/Honda'

    # HTTP request
    webpage = requests.get(URL, headers=HEADERS)
    if webpage.status_code == 200:
        print("Request successful!")
    else:
        print(f"Request failed with status code: {webpage.status_code}")
        exit()

    soup = BeautifulSoup(webpage.content, "html.parser")
    links = soup.find_all('div',attrs={'class':'gsc_col-sm-12 gsc_col-xs-12 gsc_col-md-8 listView holder posS'})
    links_list = [link.find("a", href=True).get('href') for link in links if link.find("a", href=True)]

    print(f"Found {len(links_list)} links.")

    d = {"title": [], "ex-showroom price": [],"on road price":[],"starting EMI":[], "rating": [], "reviews": []}  #"on road price":[],"starting EMI":[],

    for i, link in enumerate(links_list):
        print(f"Processing link {i+1}/{len(links_list)}: {link}")
        try:
            new_webpage = requests.get(link, headers=HEADERS)
            if new_webpage.status_code != 200:
                print(f"Failed to fetch {link}: Status {new_webpage.status_code}")
                continue
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")
            d['title'].append(get_title(new_soup))
            d['ex-showroom price'].append(get_price(new_soup))
            d['on road price'].append(get_onroadprice(link,HEADERS))
            d['starting EMI'].append(starting_emi(link,HEADERS))
            d['rating'].append(get_rating(new_soup))
            d['reviews'].append(get_review_count(new_soup))
            
            
        except Exception as e:
            print(f"Error processing link {link}: {e}")

    
    for key, value in d.items():
        print(f"Key: {key}, Length: {len(value)}")
    
    max_length = max(len(v) for v in d.values())  # Find the maximum length

    # Ensure all lists are of the same length
    for key in d:
        current_length = len(d[key])
        if current_length < max_length:
            d[key].extend([None] * (max_length - current_length))  # Pad with None
        elif current_length > max_length:
            d[key] = d[key][:max_length]  # Trim extra elements

    Tata_df = pd.DataFrame.from_dict(d)
    print(Tata_df.head())
    Tata_df['title'] = Tata_df['title'].replace('', np.nan)
    Tata_df = Tata_df.dropna(subset=['title'])

    Tata_df.to_csv("honda_df.csv", encoding='utf-8', index=False)
    print("File saved as CSV successfully!")
    
    print("File saved successfully!")


   



Request successful!
Found 5 links.
Processing link 1/5: https://www.cardekho.com/honda/city
Processing link 2/5: https://www.cardekho.com/honda/amaze
Processing link 3/5: https://www.cardekho.com/honda/elevate
Processing link 4/5: https://www.cardekho.com/honda/city-hybrid
Processing link 5/5: https://www.cardekho.com/honda/amaze-2nd-gen
Key: title, Length: 5
Key: ex-showroom price, Length: 5
Key: on road price, Length: 5
Key: starting EMI, Length: 5
Key: rating, Length: 5
Key: reviews, Length: 5
                 title      ex-showroom price  on road price  \
0           Honda City  Rs.11.82 - 16.55 Lakh  Rs.13,68,275*   
1          Honda Amaze      Rs.8 - 10.90 Lakh   Rs.8,96,579*   
2        Honda Elevate  Rs.11.69 - 16.71 Lakh  Rs.13,51,356*   
3    Honda City Hybrid     Rs.19 - 20.75 Lakh  Rs.21,78,593*   
4  Honda Amaze 2nd Gen    Rs.7.20 - 9.96 Lakh   Rs.8,13,926*   

        starting EMI rating      reviews  
0  EMI: Rs.26,040/mo    4.3  180 Reviews  
1  EMI: Rs.17,933/mo    4.6

In [19]:
# kia car df

# honda car df
# maruti suzuki df




# function to extract car name
def get_title(soup):
    try:
        title = soup.find('h1', attrs = {'class':'thcHeading'}).string.strip()
    except AttributeError:
        title = ""
    return title

# function to extract ex-showroom price
def get_price(soup):
    try:
        price = soup.find('div', class_='price').text.split('*')[0].strip()
    except AttributeError:
        price = ""
    return price

# function to extract product rating
def get_rating(soup):
    try:
        rating = soup.find('span',attrs = {'class':'ratingStarNew'}).string.strip()
    except AttributeError:
        rating = ""
    return rating

# function to extract number of user reviews
def get_review_count(soup):
    try:
        review_count = soup.find('span',attrs = {'class':'reviews'}).string.strip()
    except AttributeError:
        review_count = ""               
    return review_count


def get_onroadprice(link, HEADERS):
    try:
        
        if link == 'https://www.cardekho.com/carmodels/Toyota/Toyota_Fortuner':
            URL = 'https://www.cardekho.com/toyota-fortuner/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Toyota/Toyota_Rumion':
            URL = 'https://www.cardekho.com/toyota-rumion/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno':
            URL = 'https://www.cardekho.com/maruti-baleno/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio':
            URL = 'https://www.cardekho.com/maruti-celerio/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco':
            URL = 'https://www.cardekho.com/maruti-eeco/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis':
            URL = 'https://www.cardekho.com/maruti-ignis/car-price-in-new-delhi.htm'
        elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz':
            URL = 'https://www.cardekho.com/maruti-ciaz/car-price-in-new-delhi.htm'
        else:
            URL = link + '/price-in-new-delhi'            

        # Fetch the webpage content
        webpage = requests.get(URL, headers=HEADERS)
        webpage.raise_for_status()
        soup = BeautifulSoup(webpage.content, 'html.parser')

        # Extract the on-road price
        price_element = soup.find('td', attrs={'class': 'gsc_col-xs-4 gsc_col-md-4'})    # class="gsc_col-xs-4 gsc_col-md-4"
        if price_element:
            return price_element.text.strip()
        else:
            return "On-road price not found."

    except Exception as e:
        return f"Error occurred: {e}"

def starting_emi(link, HEADERS):

        try:
            if link == 'https://www.cardekho.com/carmodels/Toyota/Toyota_Fortuner':
                URL = 'https://www.cardekho.com/toyota-fortuner/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Toyota/Toyota_Rumion':
                URL = 'https://www.cardekho.com/toyota-rumion/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Baleno':
                URL = 'https://www.cardekho.com/maruti-baleno/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Celerio':
                URL = 'https://www.cardekho.com/maruti-celerio/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Eeco':
                URL = 'https://www.cardekho.com/maruti-eeco/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ignis':
                URL = 'https://www.cardekho.com/maruti-ignis/car-price-in-new-delhi.htm'
            elif link == 'https://www.cardekho.com/carmodels/Maruti/Maruti_Ciaz':
                URL = 'https://www.cardekho.com/maruti-ciaz/car-price-in-new-delhi.htm'
            else:
                URL = link + '/price-in-new-delhi'       
    
            # Fetch the webpage content
            webpage = requests.get(URL, headers=HEADERS)
            webpage.raise_for_status()
            soup = BeautifulSoup(webpage.content, 'html.parser')
    
            # Extract the starting EMI
            emi_element = soup.find('span', attrs={'class': 'bold'})
            if emi_element:
                return emi_element.text.strip()
            else:
                return "Starting EMI not found."

        except Exception as e:
            return f"Error occurred: {e}"






In [23]:
if __name__ == '__main__':
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en:q=0.5'
    }
    URL = 'https://www.cardekho.com/toyota-cars'

    # HTTP request
    webpage = requests.get(URL, headers=HEADERS)
    if webpage.status_code == 200:
        print("Request successful!")
    else:
        print(f"Request failed with status code: {webpage.status_code}")
        exit()

    soup = BeautifulSoup(webpage.content, "html.parser")
    links = soup.find_all('div',attrs={'class':'gsc_col-sm-12 gsc_col-xs-12 gsc_col-md-8 listView holder posS'})
    links_list = [link.find("a", href=True).get('href') for link in links if link.find("a", href=True)]

    print(f"Found {len(links_list)} links.")

    d = {"title": [], "ex-showroom price": [],"on road price":[],"starting EMI":[], "rating": [], "reviews": []}  #"on road price":[],"starting EMI":[],

    for i, link in enumerate(links_list):
        print(f"Processing link {i+1}/{len(links_list)}: {link}")
        try:
            new_webpage = requests.get(link, headers=HEADERS)
            if new_webpage.status_code != 200:
                print(f"Failed to fetch {link}: Status {new_webpage.status_code}")
                continue
            new_soup = BeautifulSoup(new_webpage.content, "html.parser")
            d['title'].append(get_title(new_soup))
            d['ex-showroom price'].append(get_price(new_soup))
            d['on road price'].append(get_onroadprice(link,HEADERS))
            d['starting EMI'].append(starting_emi(link,HEADERS))
            d['rating'].append(get_rating(new_soup))
            d['reviews'].append(get_review_count(new_soup))
            
            
        except Exception as e:
            print(f"Error processing link {link}: {e}")

    
    for key, value in d.items():
        print(f"Key: {key}, Length: {len(value)}")
    
    max_length = max(len(v) for v in d.values())  # Find the maximum length

    # Ensure all lists are of the same length
    for key in d:
        current_length = len(d[key])
        if current_length < max_length:
            d[key].extend([None] * (max_length - current_length))  # Pad with None
        elif current_length > max_length:
            d[key] = d[key][:max_length]  # Trim extra elements

    Tata_df = pd.DataFrame.from_dict(d)
    print(Tata_df.head())
    Tata_df['title'] = Tata_df['title'].replace('', np.nan)
    Tata_df = Tata_df.dropna(subset=['title'])

    Tata_df.to_csv("toyota_df.csv", encoding='utf-8', index=False)
    print("File saved as CSV successfully!")
    
    print("File saved successfully!")


   



Request successful!
Found 12 links.
Processing link 1/12: https://www.cardekho.com/carmodels/Toyota/Toyota_Fortuner
Processing link 2/12: https://www.cardekho.com/toyota/innova-crysta
Processing link 3/12: https://www.cardekho.com/toyota/land-cruiser-300
Processing link 4/12: https://www.cardekho.com/toyota/hyryder
Processing link 5/12: https://www.cardekho.com/toyota/camry
Processing link 6/12: https://www.cardekho.com/toyota/hilux
Processing link 7/12: https://www.cardekho.com/toyota/taisor
Processing link 8/12: https://www.cardekho.com/toyota/vellfire
Processing link 9/12: https://www.cardekho.com/carmodels/Toyota/Toyota_Rumion
Processing link 10/12: https://www.cardekho.com/toyota/fortuner-legender
Processing link 11/12: https://www.cardekho.com/toyota/glanza
Processing link 12/12: https://www.cardekho.com/toyota/innova-hycross
Key: title, Length: 12
Key: ex-showroom price, Length: 12
Key: on road price, Length: 12
Key: starting EMI, Length: 12
Key: rating, Length: 12
Key: reviews,

In [25]:

# List of dataset file names
file_names = ["skoda_df.csv","honda_df.csv", "toyota_df.csv", "tata_df.csv", "kia_df.csv", "hyundai_df.csv","jeep_df.csv","mahindra_df.csv","maruti_df.csv","mg_df.csv","nissan_df.csv","renault_df.csv"]

# Combine all datasets into a single DataFrame
combined_df = pd.concat([pd.read_csv(file) for file in file_names], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv("cars_df.csv", index=False)

print("All datasets have been combined and saved as cars_df.csv")

All datasets have been combined and saved as cars_df.csv
